In [1]:
from pynq import Overlay
from pynq import MMIO
from datetime import datetime

overlay = Overlay("rle.bit")
overlay.download()
print(overlay.ip_dict)

rle = MMIO(0x43C00000, 0xFFFFF)

{'SEG_decompress_0_Reg': [1136656384, 1048576, None]}


In [2]:
def bytes_to_int(x):
    
    y   = []
    num = 0
    
    # ex: 03020100
    for i, byte in enumerate(x):
        # little endian I think
        if i%4 == 0:
            num = byte
        elif i%4 == 1:
            num |= byte << 8
        elif i%4 == 2:
            num |= byte << 16
        else:
            num |= byte << 24
            y.append(int(num))
            
    if (len(x) % 4 != 0):
        # there's leftover from the above loop
        y.append(num)
        
    return y

## Read and pack bytes

In [3]:
compressed = []
data_size  = 0

# read file
file = open("weights/5-0-c.bin", "rb")
compressed = file.read()
file.close()

# pack bytes into ints
compressed = bytes_to_int(compressed)
data_size  = len(compressed)
print("in_size:", data_size*4)

in_size: 129456


## Load data

In [4]:
# set in_size
rle.write(0xC0000, data_size*4)

# load bytes into bram
for i in range(data_size):
    rle.write(0x40000 + (i*4), compressed[i])

## Start accelerator

In [5]:
# Start timer
start_sec = datetime.now().second
start_us  = datetime.now().microsecond

# Start
rle.write(0x0, 1)

# Wait for done bit
while (((rle.read(0x0) >> 1) & 0x1) == 0):
    pass

# End timer
end_sec = datetime.now().second
end_us  = datetime.now().microsecond
sec = end_sec - start_sec
us  = end_us  - start_us
print(sec, "seconds", us, "microseconds")

# Check out_size
out_size = rle.read(0xC0008)
print("out_size:", out_size)
out_size_int = int(out_size/4)

out_size: 0
0 seconds 5779 microseconds


## Verify Results

In [8]:
# byte4_mask = 0xFF000000
# byte3_mask = 0x00FF0000
# byte2_mask = 0x0000FF00
# byte1_mask = 0x000000FF

# line = 1
# # read decompressed bytes
# for i in range(out_size_int):
#     reg   = rle.read(0x80000 + (i*4))
#     byte1 = (reg & byte1_mask)
#     byte2 = (reg & byte2_mask) >> 8 
#     byte3 = (reg & byte3_mask) >> 16
#     byte4 = (reg & byte4_mask) >> 24

#     if i>0 and i%4==0:
#         print()
#         line += 1
        
#     if i%4==0:
#         print(line, " ", end='')
        
#     print("{:02X}{:02X} {:02X}{:02X} ".format(byte1, byte2, byte3, byte4), end='')

1  5F3D 902F 0000 0000 A948 4D2F 0000 0000 
2  9CE5 D232 0000 0000 916B 7457 0000 0000 
3  2CF4 AFEC 0000 0000 197E 1787 0000 0000 
4  522F B576 0000 0000 7ECC 91C2 0000 0000 
5  F3B5 5C17 0000 0000 2986 C16F 0000 0000 
6  3E55 6326 0000 0000 80E8 6467 0000 0000 
7  4074 614E 0000 0000 967D 8F9C 0000 0000 
8  1626 0521 0000 0000 B455 2E63 0000 0000 
9  9437 A0F3 0000 0000 8DC6 C16A 0000 0000 
10  1572 DCD6 0000 0000 195B 65EF 0000 0000 
11  E0B6 455E 0000 0000 9E6D B7BF 0000 0000 
12  1BA2 A5C0 0000 0000 B57C 98E5 0000 0000 
13  FF99 27B5 0000 0000 A146 4961 0000 0000 
14  7801 8544 0000 0000 A74B 5911 0000 0000 
15  7854 1E08 0000 0000 880E 7795 0000 0000 
16  AC35 9573 0000 0000 B0F9 BB07 0000 0000 
17  29E7 2F19 0000 0000 2D30 E861 0000 0000 
18  4CC0 23D0 0000 0000 6EE5 55D4 0000 0000 
19  1A75 2664 0000 0000 26E1 FD80 0000 0000 
20  5F31 5761 0000 0000 92F5 D249 0000 0000 
21  D052 151A 0000 0000 2D63 5F69 0000 0000 
22  1D81 F0F6 0000 0000 D869 7C40 0000 0000 
23  4AED 80DE 0000 